# Feature generation bitfinex bitcoin 10s data from orderbook and trades

In [7]:
from __init__ import *
from funcs_traindata import *

In [8]:
def make_data(e):
    symbol = e['symbol']
    dt = e['date']
    min_ts = e['start']
    max_ts = e['end']
    print('-------------creating features day...: ' + str(dt))
    data = pd.DataFrame({})
    try:
        data = make_features(symbol,
                             min_ts=-2000+min_ts,
                             max_ts=2000+max_ts,
                             mid_offsets=[60,120,300,600,900,1800],
                             trades_offsets=[30, 60, 120,360,900],
                             powers=[2, 4])
        data = data[~data.index.duplicated()].sort_index()
        dates = [datetime.fromtimestamp(int(d)).date() for d in data.index]
        data = data.loc[np.isin(dates,dt),:]
        print("data n : " + str(data.shape[0]))
        data.to_csv('bitfinex_btcusd/'+ symbol + '_' + str(dt)  + '.csv')
    except Exception as e:
        with open('bitfinex_btcusd/log_' + symbol + '_' + str(dt)  + '.txt', 'w') as f: f.write(str(e))
    gc.collect()

In [ ]:
WPOOL.map(make_data, get_books_days_ts(symbol='btc',date_from=date(2017,10,31),date_to=date(2017,10,31)))

In [ ]:
WPOOL = Pool(6)

if __name__ == '__main__':
    WPOOL.map(make_data, get_books_days_ts(symbol='btc',date_from=date(2017,11,1),date_to=date(2017,11,31)))
    WPOOL.close()
    client.close()
    d = merge_csvs(path = 'bitfinex_btcusd/')
    d.to_csv('training_data/bitfinex_btcusd.csv.gz', index=False,header=True,compression='gzip')
    
d = merge_csvs(path = 'bitfinex_btcusd/')
d.to_csv('training_data/bitfinex_btcusd.csv.gz', index=False,header=True,compression='gzip')

-------------creating features day...: 2017-12-01


/apps/anaconda3/lib/python3.6/site-packages/pymongo/topology.py:145: UserWarning: MongoClient opened before fork. Create MongoClient with connect=False, or create client after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#pymongo-fork-safe>
  "MongoClient opened before fork. Create MongoClient "


get book data run time: 0.7038404464721679 minutes
width and mid run time: 0.11180973052978516 minutes
offset mids run time: 0.7861955205599467 minutes
power calcs run time: 4.823688598473867 minutes
